In [ ]:
# "Ulster G2P with ICU"

> "[Incomplete]"

- toc: false
- branch: master
- hidden: true
- categories: [irish, g2p, icu, incomplete]

In [3]:
import icu

In [4]:
def transliterator_from_rules(name, rules):
    fromrules = icu.Transliterator.createFromRules(name, rules)
    icu.Transliterator.registerInstance(fromrules)
    return icu.Transliterator.createInstance(name)

In [9]:
irishlc_rules = """
:: NFD;
$uvowel=[AEIOU];
$wb=[^[:L:][:M:]];
$wb { ([nt]) } $uvowel → $1 '-';
:: lower;
:: NFC;
"""

In [10]:
irishlc = transliterator_from_rules('irishlc', irishlc_rules)

In [11]:
ulster_stress_rules = """
$wb=[^[:L:][:M:]];
$cons = [bcdfghjklmnpqrstvwxyz];
$vowel = [aeiouáéíóú];
$bvowel = [aouáóú{ae}];
$svowel = [eiéí];
:: irishlc;
$wb { anois } $wb → an'ˈ'ois;
$wb { arís } $wb → air'ˈ'ís;
$wb { isteach } $wb → ist'ˈ'each;
$wb { istigh } $wb → ist'ˈ'igh;
$wb { amach } $wb → am'ˈ'ach;
$wb $cons* { ($vowel) → 'ˈ'$1;
"""

In [12]:
stress = transliterator_from_rules('ulster_stress', ulster_stress_rules)

In [13]:
stress.transliterate("amach")

'amˈach'

In [14]:
ulster_g2p_rules = """
$wb=[^[:L:][:M:]];
$cons = [bcdfghjklmnpqrstvwxyz];
# when we transliterate past the first consonant,
# we need to use the transliteration as context
$scons = [{bʲ}cç{dʲ}{fʲ}ɟ{mʲ}{ɴʲ}ʃ{tʲ}{vʲ}{GJ}{DJ}{BJ}{MJ}];
$bcons = [{bˠ}{kˠ}x{dˠ}{fˠ}{gˠ}{mˠ}{ɴˠ}{sˠ}{tˠ}w{GH}{DH}{BH}{MH}];
$vowel = [aeiouáéíóú];
$bvowel = [aouáóú{ae}];
$svowel = [eiéí];
$apos = [’\'];
$bfce = [{á}{adh}{aidh}{aidís}{aimid}{aimis}{ainn}{as}];
$sfce = [{eá}{eadh}{idh}{idís}{imid}{imis}{inn}];

$ps = 'ˈ';
:: irishlc;
:: ulster_stress;

$wb { mb $apos? } $cons* ˈ? $bvowel → mˠ;
$wb { mb $apos? } $cons* ˈ? $svowel → mʲ;
#$bvowel $bcons* { bh → w;
#bh } $cons* ˈ? $bvowel → w;
#$svowel $scons* { bh → vʲ;
#bh } $cons* ˈ? $svowel → vʲ;
$bvowel $bcons* { bh → BH;
bh } $cons* ˈ? $bvowel → BH;
$svowel $scons* { bh → BJ;
bh } $cons* ˈ? $svowel → BJ;
bf } $bfce $wb → pˠ;
bf } $sfce $wb → pʲ;
$bvowel $bcons* { b → bˠ;
b } $cons* ˈ? $bvowel → bˠ;
$svowel $scons* { b → bʲ;
b } $cons* ˈ? $svowel → bʲ;

# gKim/Khim are things that happen too
$wb { gc } $cons* ˈ? $bvowel → gˠ;
$wb { gc } $cons* ˈ? $svowel → ɟ;
$bvowel $bcons* { ch → x;
ch } $cons* ˈ? $bvowel → x;
$svowel $scons* { ch → ç;
ch } $cons* ˈ? $svowel → ç;
$bvowel $bcons* { [ck] → kˠ;
[ck] } $cons* ˈ? $bvowel → kˠ;
$svowel $scons* { [ck] → c;
[ck] } $cons* ˈ? $svowel → c;

$wb { nd } $cons* ˈ? $bvowel → ɴˠ;
$wb { nd } $cons* ˈ? $svowel → ɴʲ;
$wb { dh $apos? } $cons* ˈ? $bvowel → ɣ;
$wb { dh $apos? } $cons* ˈ? $svowel → j;
# can't do this here, because the next pass does diphthongs
#$bvowel $bcons* { dh → ɣ;
#dh } $cons* ˈ? $bvowel → ɣ;
#$svowel $scons* { dh → j;
#dh } $cons* ˈ? $svowel → j;
$bvowel $bcons* { dh → DH;
dh } $cons* ˈ? $bvowel → DH;
$svowel $scons* { dh → DJ;
dh } $cons* ˈ? $svowel → DJ;
$wb { d $apos? } $cons* ˈ? $bvowel → dˠ;
$wb { d $apos? } $cons* ˈ? $svowel → dʲ;
$bvowel $bcons* { d → dˠ;
d } $cons* ˈ? $bvowel → dˠ;
$svowel $scons* { d → dʲ;
d } $cons* ˈ? $svowel → dʲ;
j → dʲ ;

$wb { bhf } $cons* ˈ? $bvowel → w;
$wb { bhf } $cons* ˈ? $svowel → vʲ;
fh → ;
$bvowel $bcons* { f → fˠ;
f } $cons* ˈ? $bvowel → fˠ;
$svowel $scons* { f → fʲ;
f } $cons* ˈ? $svowel → fʲ;

# can't do this here, because the next pass does diphthongs
#$bvowel $bcons* { gh → ɣ;
#gh } $cons* ˈ? $bvowel → ɣ;
#$svowel $scons* { gh → j;
#gh } $cons* ˈ? $svowel → j;
$bvowel $bcons* { gh → GH;
gh } $cons* ˈ? $bvowel → GH;
$svowel $scons* { gh → GJ;
gh } $cons* ˈ? $svowel → GJ;
$bvowel $bcons* { g → gˠ;
g } $cons* ˈ? $bvowel → gˠ;
$svowel $scons* { g → ɟ;
g } $cons* ˈ? $svowel → ɟ;

$wb { l } ˈ? $svowel → ʟʲ;
$bvowel $bcons* { ll → ʟˠ;
ll } $cons* ˈ? $bvowel → ʟˠ;
$svowel $scons* { ll → ʟʲ;
ll } $cons* ˈ? $svowel → ʟʲ;
$bvowel $bcons* { l → ʟˠ;
l } $cons* ˈ? $bvowel → ʟˠ;
$svowel $scons* { l → lʲ;
l } $cons* ˈ? $svowel → lʲ;

#$bvowel $bcons* { mh → w;
#mh } $cons* ˈ? $bvowel → w;
#$svowel $scons* { mh → vʲ;
#mh } $cons* ˈ? $svowel → vʲ;
$bvowel $bcons* { mh → MH;
mh } $cons* ˈ? $bvowel → MH;
$svowel $scons* { mh → MJ;
mh } $cons* ˈ? $svowel → MJ;
$wb { m $apos? } $cons* ˈ? $bvowel → mˠ;
$wb { m $apos? } $cons* ˈ? $svowel → mʲ;
$bvowel $bcons* { m → mˠ;
m } $cons* ˈ? $bvowel → mˠ;
$svowel $scons* { m → mʲ;
m } $cons* ˈ? $svowel → mʲ;

$wb { bp } $cons* ˈ? $bvowel → bˠ;
$wb { bp } $cons* ˈ? $svowel → bʲ;
$bvowel $bcons* { ph → fˠ;
ph } $cons* ˈ? $bvowel → fˠ;
$svowel $scons* { ph → fʲ;
ph } $cons* ˈ? $svowel → fʲ;
$bvowel $bcons* { p → pˠ;
p } $cons* ˈ? $bvowel → pˠ;
$svowel $scons* { p → pʲ;
p } $cons* ˈ? $svowel → pʲ;

$wb { r → ɾˠ;
[st]hr } ˈ? $bvowel → r̪ˠ ;
[st]hr } ˈ? $svowel → r̪ʲ ;
$bvowel $cons* { [st]hr → r̪ˠ;
[st]hr } ˈ? $bvowel → r̪ˠ ;
$svowel $cons* { [st]hr → r̪ʲ;
[st]hr } ˈ? $svowel → r̪ʲ ;

$wb { sh } ˈ? [{eá}{eái}{eoi}{eo}{iúi}{iú}] → ç ;
[st]h  → h ;
h  → h ;
$bvowel $cons* { s → sˠ;
s } $cons* ˈ? $bvowel → sˠ;
$svowel $cons* { s → ʃ;
s } $cons* ˈ? $svowel → ʃ;

$bvowel $bcons* { t → tˠ;
t } $cons* ˈ? $bvowel → tˠ;
$svowel $scons* { t → tʲ;
t } $cons* ˈ? $svowel → tʲ;

# 'oi' can represent either:
# * 'o' before a slender consonant
# * 'i' after a broad consonant
# 'goitse' can be either, I think, but abair says /i/ ([ɪ]) (I've only ever heard /o/ ([ʌ]))
$wb g $ps { oi } tse $wb → i ;
$wb an $ps { oi } s $wb → i ;

:: null;

e?a DH ai? → eː;
$ps { e?a DH → eː;
e?a DH → uː;
$ps { oi [DG]J → ai;
$ps { [ae]i DJ → eː;
$ps { a?i GJ → ai;
oi [DG]J → ə;
ai [DG]J → iː;
ei DJ → ə;
i DJ → iː;
i GJ → ə;
o MH } $wb → uː;
o MH → oː;
$ps { o BH ai? → au;
$ps { o [DG]H a? → au;
o GH a? → ə;
o BH a i? → ə;
o DH a → ə;
o DH → uː;
$ps { ea [MB]H ai → au ;
$ps { ea [MB]H a → au ;
$ps { ea [MB]H → au ;
$ps { a [MB]H ai → au ;
$ps { a [MB]H a → au ;
$ps { a [MB]H → au ;
ea [MB]H ai → ə ;
ea [MB]H a → ə ;
ea [MB]H → uː ;
a [MB]H ai → ə ;
a [MB]H a → ə ;
a [MB]H → ə ;

$ps { ei GH ea → eː;
# it seems like abair ought to have this rule, but doesn't
#a GH ai → eː;
# instead this happens (agha + i separately)
a GH ai → eːə;
a GH a → eː;
$ps { a GH → eː;
ei GH ea → ə;
a GH → ə;

$ps { eái → aː;
$ps { eá → aː;
$ps { ái → aː;
$ps { á → aː;
$ps { aei → eː;
$ps { ae → eː;
$ps { éi → eː;
$ps { é → eː;
$ps { uío → iː;
$ps { aío → iː;
$ps { aí → iː;
$ps { uí → iː;
$ps { oí → iː;
$ps { ío → iː;
$ps { aoi → iː;
$ps { ao → iː;
$ps { í → iː;
$ps { eoi → oː;
$ps { eó → oː;
$ps { eo → oː;
$ps { ió → oː;
$ps { ói → oː;
$ps { ó → oː;
eái → a;
eá → a;
ái → a;
á → a;
aei → e;
ae → e;
éi → e;
é → e;
uío → iə;
aío → iə;
aí → i;
uí → i;
oí → i;
ío → iə;
aoi → i;
ao → i;
í → i;
eoi → ɔ;
eó → ɔ;
eo → ɔ;
ió → ɔ;
ói → ɔ;
ó → ɔ;

$ps { iúi → uː;
$ps { úai → uː;
$ps { iú → uː;
iú } $wb → uː;
$ps { úi → uː;
$ps { ú → uː;
ú } $wb → uː;
iúi → u;
úai → u;
iú → u;
úi → u;
ú → u;

$ps { eai → a ;
$ps { ea → a ;
$ps { ai → a ;
$ps { a → a ;
$ps { ei → e ;
$ps { ue → e ;
$ps { e → e ;
eai → ə ;
ea → ə ;
ai → ə ;
a → ə ;
ei → ə ;
ue → ə ;
e → ə ;

$ps { oi → o ;
$ps { ui → i ;
$ps { iu → u ;
$ps { u → u ;
$ps { io → i ;
$ps { i → i ;
$wb $ps? { o } [r{ɾˠ}] → oː ;
$ps { o → o ;
oi → ə ;
ui → ə ;
iu → ə ;
io → ə ;
u → ə ;
i → ə ;
o → ə ;

DH → ɣ;
DJ → j;
GH → ɣ;
GJ → j;
BH → w;
BJ → v;
MH → w;
MJ → v;

$ps → $ps;
"""

In [15]:
ulster_g2p = transliterator_from_rules('ulster_g2p', ulster_g2p_rules)

In [17]:
ulster_g2p.transliterate("scuaibfeá")

'sˠkˠˈuəpʲa'